In [ ]:
import os
import numpy as np
import cv2
import pandas as pd
import time
from tqdm import tqdm
# from PIL import Image
import pydicom
import torch
import torch.nn as nn
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
import torch.nn
import torchvision.models as models  
import matplotlib.pyplot as plt
import pre_function as pre
import SimpleITK as sitk
# import nibabel as nib
import joblib
from torch.optim.lr_scheduler import ExponentialLR
from torch.utils.tensorboard import SummaryWriter

summaryWriter = SummaryWriter("./logs/")



In [ ]:
path0 = './segdl/0'
path1 = './segdl/1'
path_list0 = os.listdir(path0)
path_list1 = os.listdir(path1)
path_name0 = [os.path.join(path0,i) for i in path_list0]
path_name1 = [os.path.join(path1,i) for i in path_list1]
path = path_name0 + path_name1
path = [os.path.join(i,"ct_mask_new.pkl") for i in path]

len(path)

In [ ]:
means_mask, stdevs_mask,means_pet, stdevs_pet = 0,0,0,0
num_imgs = 0

for item in path:
    num_imgs += 1
    data = joblib.load(item)
    item_pet = data[1]
    pet_pixel = item_pet.astype(np.float32)
    means_pet += pet_pixel.mean()
    stdevs_pet += pet_pixel.std()

normMeanPET = means_pet / num_imgs
normStdPET = stdevs_pet / num_imgs

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

normalize_pet = transforms.Normalize(
   mean=normMeanPET,
   std=normStdPET
)

transform_pet = transforms.Compose([transforms.ToTensor(), normalize_pet])  
transform_mask = transforms.Compose([transforms.ToTensor()]) 




In [ ]:
class PLEDataset(Dataset):
    def __init__(self, data_list,shape,transform_pet,transform_mask):
    # def __init__(self, img_paths, img_labels):
        self.list = data_list
        # self.labels = labels
        self.shape = shape
        # self.size_of_images = size_of_images
        self.transform_mask = transform_mask
        self.transform_pet = transform_pet

    def __len__(self):
        return len(self.list)
    def __getitem__(self, index):
        # img = cv2.resize(self.list[index],self.size_of_images)
        path = self.list[index]
        data = joblib.load(path)
        pet = data[0]
        mask = data[1]
        num_layer = pet.shape[0]
        # if num_layer>200:
        if num_layer>387:

            pet = pet[num_layer-387:num_layer-97,:,:]
            mask = mask[num_layer-387:num_layer-97,:,:]
        # shape = [134,100,100]
        p1 = []
        p2 = []
        for i in range(3):
            if pet.shape[i] < shape[i]:
                a = int((shape[i]-pet.shape[i])/2)
                p1.append(a)
                p2.append(shape[i] - pet.shape[i] - a)
            else:
                p1.append(0)
                p2.append(0)
        img_pet = np.pad(pet,pad_width=((p1[0], p2[0]),
                                        (p1[1], p2[1]),
                                        (p1[2],p2[2])),mode="constant",constant_values=(0,0))
        img_mask = np.pad(mask,pad_width=((p1[0], p2[0]),
                                        (p1[1], p2[1]),
                                        (p1[2],p2[2])),mode="constant",constant_values=(0,0))
        c1 = abs(int((img_pet.shape[0]-shape[0])/2))
        c2 = abs(int((img_pet.shape[0]+shape[0])/2))
        h1 = abs(int((img_pet.shape[1]-shape[1])/2))
        h2 = abs(int((img_pet.shape[1]+shape[1])/2))
        w1 = abs(int((img_pet.shape[2]-shape[2])/2))
        w2 = abs(int((img_pet.shape[2]+shape[2])/2))
        img_pet = img_pet[c1:c2,h1:h2,w1:w2]
        img_mask = img_mask[c1:c2,h1:h2,w1:w2]

        img_pet = img_pet.astype(np.float32)
        img_mask = img_mask.astype(np.float32)
        TENSOR_pet = self.transform_pet(img_pet)
        TENSOR_mask = self.transform_mask(img_mask)

        return TENSOR_pet, TENSOR_mask
shape = [256,256,256]
train_path, test_path = train_test_split(path,test_size=0.2)
train_set = PLEDataset(train_path,shape,transform_pet,transform_mask)
test_set = PLEDataset(test_path,shape,transform_pet,transform_mask)
train_loader = torch.utils.data.DataLoader(train_set,batch_size=4,shuffle=True,num_workers=2,pin_memory = False)
test_loader = torch.utils.data.DataLoader(test_set,batch_size=4,shuffle=False,num_workers=2,pin_memory = False)

In [ ]:
import monai
model = monai.networks.nets.UNet(spatial_dims=3, in_channels=1, out_channels=2,channels=(32,64,128,256),strides=(2,2,2)).to(device)
# model = monai.networks.nets.BasicUNet().to(device)  # out of memory
# model = monai.networks.nets.AttentionUnet(spatial_dims=3, in_channels=1, out_channels=2, channels=(32,64,128,256),strides=(2,2,2)).to(device)  # out of memory
# model = monai.networks.nets.SwinUNETR(img_size=(128,128,128), in_channels=1, out_channels=2).to(device) # out of memory
pretrain_path = './output/newspacing_400/weights/model389.pth'
no_cuda = False
gpu_id = [0]
if not no_cuda:
    if len(gpu_id) > 1:
        model = model.cuda()
        model = nn.DataParallel(model, device_ids=gpu_id)
        net_dict = model.state_dict()
    else:
        import os
        os.environ["CUDA_VISIBLE_DEVICES"]=str(gpu_id[0])
        model = model.cuda()
        model = nn.DataParallel(model, device_ids=None)
        net_dict = model.state_dict()
else:
    net_dict = model.state_dict()

print('loading pretrained model {}'.format(pretrain_path))
pretrain = torch.load(pretrain_path)
# pretrain_dict = {k: v for k, v in pretrain['state_dict'].items() if k in net_dict.keys()}
pretrain_dict = {k: v for k, v in pretrain.items() if k in net_dict.keys()}
net_dict.update(pretrain_dict) 
model.load_state_dict(net_dict) 

print("-------- pre-train model load successfully --------")



In [ ]:
from utils import logger, weights_init, metrics, common, loss

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
# criterion = torch.nn.BCELoss().to(device)
criterion = loss.DiceLoss().to(device)
scheduler = ExponentialLR(optimizer, gamma=0.99)
num_epochs = 400

In [ ]:
len(train_loader.dataset)

In [ ]:
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
for epoch in range(num_epochs):
    start = time.time()
    per_epoch_loss = 0
    score_list = [] 
    label_list = []
    per_epoch_metrics = 0
    
    val_per_epoch_loss = 0
    val_per_epoch_metrics = 0
    
    model.train()
    with torch.enable_grad():
        for x,label in tqdm(train_loader):
            x = x.float()
            x = x.to(device)
            label = label.long()
            label = common.to_one_hot_3d(label, 2)
            label = label.to(device)
            
            # Forward pass
            x=torch.unsqueeze(x, dim=1)
            logits = model(x)
            loss = criterion(logits, label)
            # print(loss)
            per_epoch_loss += loss.item()

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # metrics = SegmentationMetric(2)
            # pred = logits.detach().numpy()
            # y = label.detach().numpy()
            # pred, y = pred.astype(np.int32), y.astype(np.int32)
            # hist = metrics.addBatch(pred, y)     
            # mIoU = metrics.meanIntersectionOverUnion() 
            # jaccard = torchmetrics.JaccardIndex(task="multiclass", num_classes=2)
            # metrics = jaccard(logits, label)   
            # metrics = monai.metrics.compute_iou(logits,label,include_background=False)[0]
            score, not_nans = monai.metrics.DiceHelper(include_background=False, sigmoid=True, softmax=True)(logits,label)
            per_epoch_metrics += score.item()
            # print(loss,score,per_epoch_loss,not_nans)
        #     pred = logits.argmax(dim=1)
        #     num_correct += torch.eq(pred, label).sum().float().item()

        # score_array = np.array(score_list)
        # label_array = np.array(label_list)
        # fpr_keras_1, tpr_keras_1, thresholds_keras_1 = roc_curve(label_array, score_array)
        # auc_keras_1 = auc(fpr_keras_1,tpr_keras_1)        

        print("Train Epoch: {}\t Loss: {:.6f}\t Acc: {}\t".format(epoch,per_epoch_loss/(2*len(train_loader)),2*per_epoch_metrics/len(train_loader.dataset)))
        summaryWriter.add_scalars('loss', {"loss":(per_epoch_loss/(2*len(train_loader)))}, epoch)
        summaryWriter.add_scalars('acc', {"acc":2*per_epoch_metrics/len(train_loader.dataset)}, epoch)

        # summaryWriter.add_scalars('auc', {"auc":auc_keras_1}, epoch)
      
  
    model.eval()
    with torch.no_grad():
        for x,label in tqdm(test_loader):
            x = x.float() 
            x = x.to(device)
            label = label.long()
            label = common.to_one_hot_3d(label, 2)
            label = label.to(device)
            #label_n = label.cpu().numpy()
            
            # val_label_list.extend(label.cpu().numpy())
            x=torch.unsqueeze(x, dim=1)
           
            # Forward pass
            logits = model(x)
            loss = criterion(logits, label)
            # print(loss)
            val_per_epoch_loss += loss.item()
            # metrics = monai.metrics.compute_iou(logits,label,include_background=False)[0]
            score, not_nans = monai.metrics.DiceHelper(include_background=False, sigmoid=True, softmax=True)(logits,label)
            val_per_epoch_metrics += score.item()
        print("val Epoch: {}\t Loss: {:.6f}\t Acc: {}\t".format(epoch,val_per_epoch_loss/(2*len(test_loader)),2*val_per_epoch_metrics/len(test_loader.dataset)))

        summaryWriter.add_scalars('loss', {"val_loss":(val_per_epoch_loss/(2*len(test_loader)))}, epoch)

        summaryWriter.add_scalars('acc', {"val_acc":2*val_per_epoch_metrics/len(test_loader.dataset)}, epoch)
        summaryWriter.add_scalars('time', {"time":(time.time() - start)}, epoch)

        
    scheduler.step()
    if (epoch+1) % 30 == 0:
        filepath = "./weights"
        folder = os.path.exists(filepath)
        if not folder:
            os.makedirs(filepath)
        path = './weights/model' + str(epoch) + '.pth'
        torch.save(model.state_dict(), path)